In [1]:
import re
import codecs
import sys
import pickle
#import time

debug=False

In [2]:
from __future__ import print_function
def warning(*objs):
    print("WARNING: ", *objs, file=sys.stderr)

In [3]:
nomFeatures="NGrams-LetterFeatures.csv"
with codecs.open(nomFeatures,"r",encoding="utf8") as stream:
    featureLines=stream.readlines()
letterFeatures={}
for featureLine in featureLines[2:]:
    elements=featureLine.strip().split("\t")
    letterFeatures[elements[0]]=[int(i) for i in elements[1:]]
#letterFeatures

In [4]:
def vectorizeNGram(ngram):
    vecteurs=[]
    for letter in ngram:
        vecteurs.extend(letterFeatures[letter])
    return vecteurs

In [5]:
#vectorizeNGram("bac")

In [6]:
nomFichier="LesMiserables.txt"
try:
    fichier = codecs.open(nomFichier, "r",encoding="utf8")
except IOError:
    warning("could not open", nomFichier)
    sys.exit()
else:
    text = fichier.readlines()
    fichier.close()

In [7]:
ngramN = 5
pc = [".", ",", ";", ":", "?", "!", "--", "...", u"«", u"»", "(", ")", "_","*",'"',"/","+"]

In [8]:
def normaliserTexte(text,nospace=False):
    normalText=[]
    for line in text:
        for ponct in pc: 
            line=line.replace(ponct,"")
        line=line.replace("'"," ")
        line=line.replace("-"," ")
        if nospace:
            line=re.sub('\s+','',line)
        else:
            line=re.sub('\s+',' ',line)
        normalText.append(line.lower())
    return normalText

def extraireNgrams(text,noclasses=False):
    ngrams=[]
    classes=[]
    for line in text[:]:
        line = line.strip()
    #    print (line)
        espace = line.count(" ")
#        print (len(line),ngramN,espace)
        i = 0
        while i <= (len(line) - ngramN - espace):
            if line[i] != " ":
                lettres = 0
                lettreslocales = 0
                ngramLocal=[]
                j = 0
                type = ""
                while lettres < ngramN:
                    if line[i + j] != " ":
                        ngramLocal.append(line[i + j])
                        lettres += 1
                        lettreslocales += 1
                    else:
                        type = type + str(lettreslocales) + "_"
                        lettreslocales = 0
                    j += 1
                type = type + str(lettreslocales)
                ngrams.append(ngramLocal)
                classes.append(type)
            i = i + 1
    if noclasses:
        return ngrams
    else:
        return(ngrams, classes)

In [9]:
trainTexte = normaliserTexte(text)
(ngramTrain,classesTrain)=extraireNgrams(trainTexte)
print (ngramTrain[0],classesTrain[0])

[u't', u'o', u'm', u'e', u'i'] 4_1


In [10]:
ngramTrain2=[]
for n,ngram in enumerate(ngramTrain):
    if debug: print (n, "".join(ngram),end=", ")
    ngramTrain2.append(vectorizeNGram(ngram))
    
#ngramTrain2[:10]

In [11]:
from sklearn import svm
#clf = svm.SVC(gamma=0.001, C=100.)
clf = svm.SVC()

In [ ]:
%%time
clf.fit(ngramTrain2, classesTrain)

In [ ]:
#testSource=[u"Dans cette histoire, il y a un loup, une chêvre, et un agneau. Tout se passe bien tant que le loup et l'agneau ne sont pas seuls."]

In [ ]:
nomFichier="texte.txt"
try:
    fichier = codecs.open(nomFichier, "r", encoding="utf-8")
except IOError:
    warning("could not open", nomFichier)
    sys.exit()
else:
    testSource = fichier.readlines()
    fichier.close()

In [ ]:
testTexte=normaliserTexte(testSource,True)
ngramTest=extraireNgrams(testTexte,True)
print (ngramTest[0])

In [ ]:
ngramTest2=[]
for n,ngram in enumerate(ngramTest):
    if n>2500000: print (n, "".join(ngram),end=", ")
    ngramTest2.append(vectorizeNGram(ngram))
    
#ngramTrain2[:10]

In [ ]:
prediction=clf.predict(ngramTest2)

In [ ]:
resultChunks=[]
for i,element in enumerate(prediction):
    chunks=[int(n) for n in element.split("_")]
#    print (chunks)
    debut=0
    elementChunks=[]
    for chunk in chunks:
#        print (chunk,ngramTest[i][debut:debut+chunk],end=",")
#    print ()
        elementChunks.append(ngramTest[i][debut:debut+chunk])
#        print (ngramTest[i][debut:debut+chunk],end=" ")
        debut+=chunk
#    print ()
    resultChunks.append(elementChunks)
#print (resultChunks)

In [ ]:
testSpaces={}
for i,chunks in enumerate(resultChunks):
    posChunk=0
    for chunk in chunks:
        if posChunk!=0: 
#            print (i+posChunk,chunk)
            if not i+posChunk in testSpaces:
                testSpaces[i+posChunk]=0
            testSpaces[i+posChunk]+=1
        posChunk+=len(chunk)
#testSpaces

In [ ]:

for i,lettre in enumerate(testTexte):
#    print (i)
    if i in testSpaces:
        # choisir le seuil d'apparition des espaces
        # au dessus de deux types convergents : >2
        # on insère un espace
#        print (testSpaces[i])        
        if testSpaces[i]>2:
            print ("_",end="")
        # sinon, par exemple, on colle les lettres
        # on pourrait aussi mettre le nombre de votes convergents
        # pour voir ce que ça donne
        else:
            pass
#            print (testSpaces[i],end="")
    print (lettre,end="")
#    time.sleep(1)

In [ ]:
testTexte

In [ ]:
prediction=clf.predict(skTest)

In [ ]:
ngramComplete=ngramTrain+ngramTest
textComplete=["".join(x) for x in ngramComplete]
textTrain=["".join(x) for x in ngramTrain]
textTest=["".join(x) for x in ngramTest]
print (len(textComplete),len(textTrain),len(textTest))
print (textComplete[0],textTrain[0],textTest[0])

CountVectorizer permet de traiter le texte comme des données numériques  
- analyzer permet de définir le niveau de tokenization (word, char)
- ngram_range permet de définir les ngrams qui seront extraits (min, max)

La méthode fit_transform permet de déterminer les données numériques (features) qui seront utilisées pour représenter le texte.

Ensuite, on peut utiliser la méthode transform pour représenter un texte en utilisant ces features.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
transformVectors = CountVectorizer(analyzer=u'char',ngram_range=(5, 5))
skComplete = transformVectors.fit_transform(textComplete)

SVM est une des formes d'apprentissage
- gamma définit le nombre de features qui seront utilisées pour l'apprentissage
- C ???

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

skTrain est la représentation numérique de textTrain

clf.fit permet de faire l'apprentissage du lien entre les données de skTrain et les étiquettes fournies dataClasses

In [ ]:
%%time
skTrain=transformVectors.transform(textTrain)
clf.fit(skTrain, classesTrain)

In [ ]:
with open('SVC-Decoupages.pkl', 'wb') as output:
   pickle.dump(clf, output, pickle.HIGHEST_PROTOCOL)

skTest est la représentation numérique de textTest

clf.predict permet de faire la prédiction des étiquettes pour les données de skTest

In [ ]:
skTest=transformVectors.transform(textTest)
prediction=clf.predict(skTest)

##interprétation des résultats

###Découpage des ngrams

découpage des ngrams de test en fonction des étiquettes => resultChunks

In [ ]:
resultChunks=[]
for i,element in enumerate(prediction):
    chunks=[int(n) for n in element.split("_")]
#    print (chunks)
    debut=0
    elementChunks=[]
    for chunk in chunks:
        elementChunks.append(textTest[i][debut:debut+chunk])
#        print (textTest[i][debut:debut+chunk],end=" ")
        debut+=chunk
#    print ()
    resultChunks.append(elementChunks)
#print (resultChunks)

###Calcul des votes

calcul des votes pour les séparations entre mots

In [ ]:
testSpaces={}
for i,chunks in enumerate(resultChunks):
    posChunk=0
    for chunk in chunks:
        if posChunk!=0: 
#            print (i+posChunk,chunk)
            if not i+posChunk in testSpaces:
                testSpaces[i+posChunk]=0
            testSpaces[i+posChunk]+=1
        posChunk+=len(chunk)
#testSpaces

###Découpage de test

Séparation des mots

In [ ]:
for i,lettre in enumerate(testTexte[200:250]):
    if i in testSpaces:
        # choisir le seuil d'apparition des espaces
        # au dessus de deux types convergents : >2
        # on insère un espace
        if testSpaces[i]>2:
            print ("_",end="")
        # sinon, par exemple, on colle les lettres
        # on pourrait aussi mettre le nombre de votes convergents
        # pour voir ce que ça donne
        else:
            pass
#            print (testSpaces[i],end="")
    print (lettre,end="")
#    time.sleep(1)